In [1]:
import pandas as pd
import numpy as np

df = pd.read_json('analyzed_comments.json')

In [2]:
def normalize_rate(r):
    if r == 5:
        return 1
    elif r == 4:
        return 0
    else:
        return -1

def build_corpus(comments):
    X, y = [], []

    for _i, comment in comments.iterrows():
        if comment['text']:
            X.append(comment['text'])
            y.append(normalize_rate(comment['rate']))

    return X, y

In [3]:
X, y = build_corpus(df)

In [4]:
from collections import Counter

stats = Counter(y)

print('Positive: ', stats[1])
print('Neutral: ', stats[0])
print('Negative: ', stats[-1])
print('Total: ', len(X))

len(X) == len(y)

Positive:  1051
Neutral:  407
Negative:  371
Total:  1829


True

## Базове рішення

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

def top_features(vectorizer, clf, n):
    """Prints features with the highest coefficient values, per class"""
    feature_names = vectorizer.get_feature_names()
    for i, class_label in enumerate(clf.classes_):
        top = np.argsort(clf.coef_[i])
        reversed_top = top[::-1]
        print("%s: %s\n" % (class_label,
              ', '.join(feature_names[j] for j in reversed_top[:n])))        

def cross_val_report(clf, x, y):
    scores = cross_val_score(clf, x, y, cv=5, scoring='f1_macro')
    print('Scores: ', scores)
    print('Mean: ', scores.mean())

def class_report(y_test, y_pred): 
    target_names = ['Negative', 'Neutral', 'Positive']
    print(classification_report(y_test, y_pred, target_names=target_names))

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1789)
print(Counter(y_train), Counter(y_test))

Counter({1: 779, 0: 306, -1: 286}) Counter({1: 272, 0: 101, -1: 85})


In [8]:
vec = CountVectorizer()
vec.fit_transform(X_train)
X_features = vec.transform(X_train)

clf = MultinomialNB()

cross_val_report(clf, X_features, y_train)

Scores:  [0.52360465 0.52999074 0.47845898 0.51672058 0.42563423]
Mean:  0.4948818347519178


In [9]:
clf = MultinomialNB()
clf.fit(X_features, y_train)

X_features_test = vec.transform(X_test)
y_pred = clf.predict(X_features_test)

In [10]:
confusion_matrix(y_test, y_pred)

array([[ 39,   4,  42],
       [  8,   5,  88],
       [  9,   5, 258]])

In [11]:
class_report(y_test, y_pred)

              precision    recall  f1-score   support

    Negative       0.70      0.46      0.55        85
     Neutral       0.36      0.05      0.09       101
    Positive       0.66      0.95      0.78       272

    accuracy                           0.66       458
   macro avg       0.57      0.49      0.47       458
weighted avg       0.60      0.66      0.59       458



In [12]:
top_features(vec, clf, 20)

-1: не, на, що, але, за, як, це, до, при, так, для, він, працює, по, після, бойлер, та, все, через, дуже

0: не, на, що, за, але, для, працює, до, як, та, при, це, добре, все, якщо, дуже, води, так, він, повітря

1: на, не, що, дуже, за, працює, для, все, як, та, але, це, швидко, до, повітря, вже, бойлер, зволожувач, води, без



## Спостереження
- Багато false-positive щодо позитивності відгука. Це може відбуватися через те що: 1) в даних є нерівномірний розподіл оцінок (позитивних відгуків набаго більше ніж усіх інших); 2) я визначив, що відугки з оцінкою 4 — нейтральні, та 3 — негативні. Але відгуки з такими оцінками доволі змішані за своїм сентиментом; 3) замала кількість негативних відгуків (286)
- Модель погано роспізнає нейтральні відгуки. Категорію нейтрального відгука було важко визначити. Тому очікувано такі нізьки результати

## Балансування классів

In [242]:
shuffled_df = df.sample(frac=1,random_state=4)
less_than_five_df = shuffled_df.loc[shuffled_df['rate'] != 5]
rate_five_df = shuffled_df.loc[shuffled_df['rate'] == 5].sample(n=500,random_state=1089)
normalized_df = pd.concat([rate_five_df, less_than_five_df])

In [243]:
X, y = build_corpus(normalized_df)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1789)

In [244]:
print(Counter(y_train), Counter(y_test))

Counter({1: 353, 0: 309, -1: 273}) Counter({1: 116, -1: 98, 0: 98})


In [245]:
vec = CountVectorizer()
vec.fit_transform(X_train)
X_features = vec.transform(X_train)

clf = MultinomialNB()

cross_val_report(clf, X_features, y_train)

Scores:  [0.57327582 0.57043221 0.5688628  0.59140831 0.49135836]
Mean:  0.5590674999057608


In [246]:
clf = MultinomialNB()
clf.fit(vec.transform(X_train), y_train)

X_features_test = vec.transform(X_test)
y_pred = clf.predict(X_features_test)

In [247]:
confusion_matrix(y_test, y_pred)

array([[55, 22, 21],
       [11, 46, 41],
       [11, 38, 67]])

In [248]:
class_report(y_test, y_pred)

              precision    recall  f1-score   support

    Negative       0.71      0.56      0.63        98
     Neutral       0.43      0.47      0.45        98
    Positive       0.52      0.58      0.55       116

    accuracy                           0.54       312
   macro avg       0.56      0.54      0.54       312
weighted avg       0.55      0.54      0.54       312



In [130]:
top_features(vec, clf, 20)

-1: не, на, що, як, але, за, це, до, так, при, через, працює, по, він, бойлер, для, після, та, його, все

0: не, на, що, за, але, для, до, працює, як, при, це, дуже, та, все, води, добре, вже, коли, по, тому

1: на, не, що, дуже, за, працює, для, як, та, це, повітря, все, швидко, бойлер, але, при, до, зволожувач, коли, рекомендую



## Спостереження
- Ситуація дещо покращилась
- Близько половини нейтральних відгуків роспізнається як позитивні
- Також багато негативних відгуків роспізнається як нейтральні та позитивні (можливо через включення оцінки 3 до негативного классу)
- Якщо розподілити відгуки з оцінками 3 і 4 більш точно, то ситуація може покращитись

## Додаткова обробка даних

In [252]:
def has_positive_tone(text):
    if re.search('\добре|гарний|хороший|чудовий|тихий\b', text, re.I):
        # print('- ' + text)
        return True

def has_negative_tone(text):
    if re.search('не (рекомендую|приємний|гарантійний|раджу|сподобалось)', text, re.I) or \
       re.search('\bпоганий|bпогана|жахливий|на жаль\b', text, re.I):
        # print('- ' + text)
        return True

def normalize_rate_v2(comment):
    if comment['rate'] == 5:
        return 1
    elif comment['rate'] == 3 or comment['rate'] == 4:
        if has_negative_tone(comment['text']):
            return 1
        elif has_positive_tone(comment['text']):
            return -1
        else:
            return 0
    else:
        return -1
        
X, y = [], []

for _i, comment in normalized_df.iterrows():
    if comment['text']:
        X.append(comment['text'])
        y.append(normalize_rate_v2(comment))
        
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1789)

print(Counter(y_train), Counter(y_test))

Counter({1: 357, 0: 317, -1: 261}) Counter({1: 118, 0: 102, -1: 92})


In [253]:
vec = CountVectorizer()
vec.fit_transform(X_train)
X_features = vec.transform(X_train)

clf = MultinomialNB()

cross_val_report(clf, X_features, y_train)

Scores:  [0.52288186 0.52702742 0.54819728 0.5314509  0.50912945]
Mean:  0.527737382709492


In [254]:
clf = MultinomialNB()
clf.fit(vec.transform(X_train), y_train)

X_features_test = vec.transform(X_test)
y_pred = clf.predict(X_features_test)

In [255]:
confusion_matrix(y_test, y_pred)

array([[45, 16, 31],
       [24, 40, 38],
       [16, 30, 72]])

In [256]:
class_report(y_test, y_pred)

              precision    recall  f1-score   support

    Negative       0.53      0.49      0.51        92
     Neutral       0.47      0.39      0.43       102
    Positive       0.51      0.61      0.56       118

    accuracy                           0.50       312
   macro avg       0.50      0.50      0.50       312
weighted avg       0.50      0.50      0.50       312



## Спостереження

- Розподілення неоднозначних відгуків за правилами спрацювало гірше ніж я очікував.
- Негативні відгуки стали гірше розпізнаватись, та збільшилсь їх false-positive частка. Мабуть бо їх стало менше.
- Нейтальні відгуки стали рідше відзначатися (бо їх стало меньше)
- Позитивні відгуки стали розпізнаватись краще.
- За f_macro зміни погіршили точність моделі

## Біграми

In [265]:
X, y = build_corpus(normalized_df)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1789)

vec = CountVectorizer(ngram_range=(2, 2))
vec.fit_transform(X_train)
X_features = vec.transform(X_train)

clf = MultinomialNB()

cross_val_report(clf, X_features, y_train)

Scores:  [0.55390983 0.5292769  0.49184329 0.5067558  0.44769504]
Mean:  0.5058961702852095


In [266]:
clf = MultinomialNB()
clf.fit(vec.transform(X_train), y_train)

X_features_test = vec.transform(X_test)
y_pred = clf.predict(X_features_test)

In [267]:
confusion_matrix(y_test, y_pred)

array([[51, 27, 20],
       [15, 39, 44],
       [ 7, 34, 75]])

In [268]:
class_report(y_test, y_pred)

              precision    recall  f1-score   support

    Negative       0.70      0.52      0.60        98
     Neutral       0.39      0.40      0.39        98
    Positive       0.54      0.65      0.59       116

    accuracy                           0.53       312
   macro avg       0.54      0.52      0.53       312
weighted avg       0.54      0.53      0.53       312



In [269]:
top_features(vec, clf, 20)

-1: не рекомендую, не працює, так як, він не, це не, тому що, те що, не знаю, не раджу, сказав що, до цього, так що, нічого не, не було, його не, як на, пів року, не так, будь ласка, гарантійний талон

0: так як, поки що, за таку, не було, таку ціну, він не, працює тихо, не дуже, те що, тому що, але це, працює добре, на стіну, ціна якість, свою ціну, гарантійний талон, працює вже, це не, що не, даний момент

1: те що, під час, досить швидко, так як, за таку, поки що, дуже зручно, сказати що, хороший бойлер, гарячої води, все працює, працює на, працює тихо, не було, свої гроші, що це, таку ціну, дуже задоволена, своїх грошей, працює добре



## Спостереження

Використання біграм зменшило якість розпізнавання нейтральни та негативних відгуків

## Леми

In [ ]:
import stanza
nlp = stanza.Pipeline(lang='uk', processors='tokenize,mwt,pos,lemma')

In [ ]:
def lemma_tokenizer(text):
    doc = nlp(text)
    return [w.lemma for s in doc.sentences for w in s.words]

vec = CountVectorizer(tokenizer=lemma_tokenizer)
vec.fit_transform(X_train)

stanza виявилась дуууже повільною

In [270]:
import tokenize_uk
import pymorphy2
import re

morph = pymorphy2.MorphAnalyzer(lang='uk')


def lemma_tokenizer_v2(text):
    tokens = tokenize_uk.tokenize_uk.tokenize_words(text)
    return [morph.parse(t)[0].normal_form for t in tokens if t.isalpha()]

In [271]:
vec = CountVectorizer(tokenizer=lemma_tokenizer_v2)
vec.fit_transform(X_train)

X_features = vec.transform(X_train)

clf = MultinomialNB()

cross_val_report(clf, X_features, y_train)

Scores:  [0.56140285 0.56368788 0.54405514 0.5791593  0.50381395]
Mean:  0.5504238240015236


In [272]:
vec = CountVectorizer(tokenizer=lemma_tokenizer_v2)
vec.fit_transform(X_train)

X_features = vec.transform(X_train)

clf.fit(vec.transform(X_train), y_train)

X_features_test = vec.transform(X_test)
y_pred = clf.predict(X_features_test)

In [273]:
confusion_matrix(y_test, y_pred)

array([[58, 23, 17],
       [14, 43, 41],
       [ 9, 30, 77]])

In [274]:
class_report(y_test, y_pred)

              precision    recall  f1-score   support

    Negative       0.72      0.59      0.65        98
     Neutral       0.45      0.44      0.44        98
    Positive       0.57      0.66      0.61       116

    accuracy                           0.57       312
   macro avg       0.58      0.56      0.57       312
weighted avg       0.58      0.57      0.57       312



In [275]:
top_features(vec, clf, 20)

-1: не, на, в, і, що, вода, бути, з, він, як, але, працювати, за, а, у, цей, бойлер, це, до, так

0: не, на, і, в, що, бути, з, за, але, вода, для, працювати, до, у, а, весь, температура, кімната, як, перти

1: і, в, на, не, бути, що, з, дуже, працювати, за, вода, у, весь, бойлер, як, задоволений, для, та, це, повітря



## Спостереження

- Використання лем трохи зменшило кільксть false-positive для усіх классів

## Леми + біграми

In [276]:
vec = CountVectorizer(ngram_range=(2, 2), tokenizer=lemma_tokenizer_v2)
vec.fit_transform(X_train)

X_features = vec.transform(X_train)

clf = MultinomialNB()

cross_val_report(clf, X_features, y_train)

Scores:  [0.49977632 0.56007659 0.53709075 0.52988215 0.43768312]
Mean:  0.5129017880732315


In [277]:
vec = CountVectorizer(ngram_range=(2, 2), tokenizer=lemma_tokenizer_v2)
vec.fit_transform(X_train)

X_features = vec.transform(X_train)

clf.fit(vec.transform(X_train), y_train)

X_features_test = vec.transform(X_test)
y_pred = clf.predict(X_features_test)

In [278]:
confusion_matrix(y_test, y_pred)

array([[51, 26, 21],
       [19, 32, 47],
       [10, 35, 71]])

In [279]:
class_report(y_test, y_pred)

              precision    recall  f1-score   support

    Negative       0.64      0.52      0.57        98
     Neutral       0.34      0.33      0.34        98
    Positive       0.51      0.61      0.56       116

    accuracy                           0.49       312
   macro avg       0.50      0.49      0.49       312
weighted avg       0.50      0.49      0.49       312



In [280]:
top_features(vec, clf, 20)

-1: не рекомендувати, не працювати, в кімната, що в, так як, він не, той що, і не, сказати що, кв м, в мен, в цей, так і, я не, том що, не знати, гарантійний талон, це не, гарячий вода, сервісний центр

0: за такий, так як, в кімната, і не, поки що, не бути, за свій, такий ціна, він не, кв м, той що, працювати тихо, що бути, в комплект, не дуже, на стіна, вода в, працювати добре, том що, в інший

1: той що, в кімната, кв м, дуже задоволений, під час, гарячий вода, за такий, сказати що, у ми, досить швидко, не бути, і не, в квартира, так як, поки що, у мен, дуже сподобатися, задоволений покупка, дуже зручно, не мати



## Спостереження

- Використання біграм + лем зменшило якість розпізнавання негативних та нейтральних классів